# 範例四_台灣證券交易所 
https://www.twse.com.tw/zh/  

In [2]:
import numpy as np
import requests
import pandas as pd
import datetime

#   http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=20180817&stockNo=2330  取一個月的股價與成交量
def get_stock_history(date, stock_no):
    quotes = []
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=%s&stockNo=%s' % ( date*10+1, stock_no)
    print("連結網址 : ",url)
    r = requests.get(url)
    data = r.json()
    return transform(data['data'])  #進行資料格式轉換

def transform_date(date):
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d  #民國轉西元
    
def transform_data(data):
    data[0] = datetime.datetime.strptime(transform_date(data[0]), '%Y/%m/%d')
    data[1] = int(data[1].replace(',', ''))  #把千進位的逗點去除
    data[2] = int(data[2].replace(',', ''))
    data[3] = float(data[3].replace(',', ''))
    data[4] = float(data[4].replace(',', ''))
    data[5] = float(data[5].replace(',', ''))
    data[6] = float(data[6].replace(',', ''))
    data[7] = float(0.0 if data[7].replace(',', '') == 'X0.00' else data[7].replace(',', ''))  # +/-/X表示漲/跌/不比價
    data[8] = int(data[8].replace(',', ''))
    return data

def transform(data):
    return [transform_data(d) for d in data]

def create_df(date,stock_no):
    s = pd.DataFrame(get_stock_history(date, stock_no))
    s.columns = ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
                
    stock = []
    for i in range(len(s)):
        stock.append(stock_no)
    s['公司代號'] = pd.Series(stock ,index=s.index)  #新增股票代碼欄，之後所有股票進入資料表才能知道是哪一張股票
    datelist = []
    for i in range(len(s)):
        datelist.append(s['日期'][i])
    s.index = datelist  #索引值改成日期
    s2 = s.drop(['日期'],axis = 1)  #刪除日期欄位
    mlist = []
    for item in s2.index:
        mlist.append(item.month)
    s2['月份'] = mlist  #新增月份欄位
    return s2
  

In [107]:
year = 2020 #改這裡 年
stock = 2330 #改這裡，公司代號
result = pd.DataFrame(columns=['成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數','公司代號','月份'])
for month in range(1,12):
    try :
        temp_df = create_df(year*100+month, stock)
        result = pd.concat([result,temp_df])
    except:
        break

result.head(1)     

連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020011&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020021&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020031&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020041&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020051&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020061&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020071&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020081&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020091&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020101&stockNo=2330
連結網址 :  http://www.twse.com.tw/exchangeReport/STOCK_DAY?date=2020111&stockNo=2330


,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,公司代號,月份
2020-01-02,33282120,11224165450,332.5,339.0,332.5,339.0,8.0,17160,2330,1


In [108]:
df = result
df .head(1)

,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,公司代號,月份
2020-01-02,33282120,11224165450,332.5,339.0,332.5,339.0,8.0,17160,2330,1


In [109]:
if 'level_0' in df.columns:
    df.reset_index(inplace=True,drop=True)
else:
    df.reset_index(inplace=True)
df = df.rename(columns={"index": "時間"})
if 'level_0' in df.columns:
    df = df.drop(['level_0'],axis = 1)

df.to_csv('./Data/{0}_{1}_台灣證券交易所.csv'.format(stock,year), encoding='utf_8_sig')  

df 

,時間,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數,公司代號,月份
0,2020-01-02,33282120,11224165450,332.5,339.0,332.5,339.0,8.0,17160,2330,1
1,2020-01-03,42023268,14295651580,344.0,345.0,335.5,339.5,0.5,20545,2330,1
2,2020-01-06,45677057,15210531318,333.0,334.5,332.0,332.0,-7.5,22944,2330,1
3,2020-01-07,51746181,17053282011,332.5,333.0,326.5,329.5,-2.5,22362,2330,1
4,2020-01-08,37913748,12484410088,325.0,333.0,325.0,329.5,0.0,18228,2330,1
...,...,...,...,...,...,...,...,...,...,...,...
204,2020-11-05,36539446,16404944484,451.5,451.5,445.5,451.0,1.0,24764,2330,11
205,2020-11-06,32644190,14788989058,455.0,455.5,450.0,452.5,1.5,20999,2330,11
206,2020-11-09,43415670,19865414906,458.0,460.0,454.0,458.5,6.0,27868,2330,11
207,2020-11-10,36786882,16610287996,452.0,454.5,448.5,451.0,-7.5,31009,2330,11


#  股價波動

# 黃金/死亡交叉KD線(圖)

# 整體V.S.單間 